In [1]:
from pyspark.sql import SparkSession

spark = (SparkSession.builder
         .master("local[*]")
         .appName('Example')
         .getOrCreate()
        )
# Где "*" в "local[ * ]" обозначает использование всех доступных ядер процессора.

spark.stop()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/28 00:19:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
from pyspark.sql import SparkSession

spark = (SparkSession.builder
         .master("local[*]")
         .appName('Example')
         .getOrCreate()
        )
# Создание RDD
data = [("Вася", 30), ("Петя", 24), ("Маша", 35), ("Даша", 29)]
rdd = spark.sparkContext.parallelize(data)

# Преобразование RDD в DataFrame
df_from_rdd = rdd.toDF(["Name", "Age"])
# Выводим схему
df_from_rdd.printSchema()

# Останавливаем SparkSession
spark.stop()

root
 |-- Name: string (nullable = true)
 |-- Age: long (nullable = true)



In [3]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

spark = (SparkSession.builder
         .master("local[*]")
         .appName('Example')
         .getOrCreate()
        )

data = [("Алиса", 30, "Москва"), ("Вася", 25, "Казань"), ("Петя", 35, "Новосибирск"), ("Маша", 32, "Санкт-Петербург"), ("Артем", 23, "Москва"), ("Виталий", 35, "Махачкала"), ("Маша", 18, "Казань")]
df_data = spark.createDataFrame(data, ["name", "age", "city"])

df_data.show()


+-------+---+---------------+
|   name|age|           city|
+-------+---+---------------+
|  Алиса| 30|         Москва|
|   Вася| 25|         Казань|
|   Петя| 35|    Новосибирск|
|   Маша| 32|Санкт-Петербург|
|  Артем| 23|         Москва|
|Виталий| 35|      Махачкала|
|   Маша| 18|         Казань|
+-------+---+---------------+



In [4]:
select_df = df_data.select("name", "age")
select_df.show()

+-------+---+
|   name|age|
+-------+---+
|  Алиса| 30|
|   Вася| 25|
|   Петя| 35|
|   Маша| 32|
|  Артем| 23|
|Виталий| 35|
|   Маша| 18|
+-------+---+



In [5]:
select_expr = df_data.selectExpr("upper(name) as upper_name", "age * 365 as age_in_days_expr")
select_expr.show()

+----------+----------------+
|upper_name|age_in_days_expr|
+----------+----------------+
|     АЛИСА|           10950|
|      ВАСЯ|            9125|
|      ПЕТЯ|           12775|
|      МАША|           11680|
|     АРТЕМ|            8395|
|   ВИТАЛИЙ|           12775|
|      МАША|            6570|
+----------+----------------+



In [6]:
new_column_df = df_data.withColumn("age_in_days", df_data.age * 365)
new_column_df.show()

+-------+---+---------------+-----------+
|   name|age|           city|age_in_days|
+-------+---+---------------+-----------+
|  Алиса| 30|         Москва|      10950|
|   Вася| 25|         Казань|       9125|
|   Петя| 35|    Новосибирск|      12775|
|   Маша| 32|Санкт-Петербург|      11680|
|  Артем| 23|         Москва|       8395|
|Виталий| 35|      Махачкала|      12775|
|   Маша| 18|         Казань|       6570|
+-------+---+---------------+-----------+



In [7]:
multiple_columns_df = df_data.withColumns({
    "age_in_days": F.col("age") * 365,
    "is_adult": F.col("age") >= 18,
    "status": F.lit("Active") # Добавляем строковый литерал
})
multiple_columns_df.show()

+-------+---+---------------+-----------+--------+------+
|   name|age|           city|age_in_days|is_adult|status|
+-------+---+---------------+-----------+--------+------+
|  Алиса| 30|         Москва|      10950|    true|Active|
|   Вася| 25|         Казань|       9125|    true|Active|
|   Петя| 35|    Новосибирск|      12775|    true|Active|
|   Маша| 32|Санкт-Петербург|      11680|    true|Active|
|  Артем| 23|         Москва|       8395|    true|Active|
|Виталий| 35|      Махачкала|      12775|    true|Active|
|   Маша| 18|         Казань|       6570|    true|Active|
+-------+---+---------------+-----------+--------+------+



In [8]:
filter_df = df_data.where(df_data.city == "Москва")
filter_df.show()

+-----+---+------+
| name|age|  city|
+-----+---+------+
|Алиса| 30|Москва|
|Артем| 23|Москва|
+-----+---+------+



In [9]:
sort_df = df_data.sort(df_data.age.desc(), df_data.name.asc())
sort_df.show()

+-------+---+---------------+
|   name|age|           city|
+-------+---+---------------+
|Виталий| 35|      Махачкала|
|   Петя| 35|    Новосибирск|
|   Маша| 32|Санкт-Петербург|
|  Алиса| 30|         Москва|
|   Вася| 25|         Казань|
|  Артем| 23|         Москва|
|   Маша| 18|         Казань|
+-------+---+---------------+



In [10]:
# Только уникальные строки
distinct_df = df_data.distinct()
print("Результат df.distinct() (уникальные строки):")
distinct_df.show()

# Удаление дубликатов только по столбцу name
drop_duplicates_df = df_data.dropDuplicates(["name"])
print("Результат df.dropDuplicates(['name']) (уникальные по 'name'):")
drop_duplicates_df.show()


Результат df.distinct() (уникальные строки):
+-------+---+---------------+
|   name|age|           city|
+-------+---+---------------+
|  Алиса| 30|         Москва|
|   Вася| 25|         Казань|
|   Петя| 35|    Новосибирск|
|   Маша| 32|Санкт-Петербург|
|  Артем| 23|         Москва|
|Виталий| 35|      Махачкала|
|   Маша| 18|         Казань|
+-------+---+---------------+

Результат df.dropDuplicates(['name']) (уникальные по 'name'):
+-------+---+---------------+
|   name|age|           city|
+-------+---+---------------+
|  Алиса| 30|         Москва|
|  Артем| 23|         Москва|
|   Вася| 25|         Казань|
|Виталий| 35|      Махачкала|
|   Маша| 32|Санкт-Петербург|
|   Петя| 35|    Новосибирск|
+-------+---+---------------+



In [11]:
# Добавляем новый df
timezone = [("Москва", 3), ("Махачкала", 3), ("Казань", 3), ("Новосибирск", 7), ("Петербург", 3)]
df_timezone = spark.createDataFrame(timezone, ["city", "timezone"])

joined_df = df_timezone.join(df_data, df_data.city == df_timezone.city, "inner")
joined_df.show()

+-----------+--------+-------+---+-----------+
|       city|timezone|   name|age|       city|
+-----------+--------+-------+---+-----------+
|     Казань|       3|   Вася| 25|     Казань|
|     Казань|       3|   Маша| 18|     Казань|
|  Махачкала|       3|Виталий| 35|  Махачкала|
|     Москва|       3|  Алиса| 30|     Москва|
|     Москва|       3|  Артем| 23|     Москва|
|Новосибирск|       7|   Петя| 35|Новосибирск|
+-----------+--------+-------+---+-----------+



In [12]:
# DataFrame 2 (имеет общие строки с df_data и новые)
data2 = [
    ("Олег", 28, "Самара"),
    ("Вася", 25, "Казань"), # Дубликат 
    ("Петя", 35, "Новосибирск"), # Дубликат 
    ("Елена", 40, "Москва")
]
df2 = spark.createDataFrame(data2, ["name", "age", "city"])

union_result_df = df_data.union(df2)
union_result_df.show()


+-------+---+---------------+
|   name|age|           city|
+-------+---+---------------+
|  Алиса| 30|         Москва|
|   Вася| 25|         Казань|
|   Петя| 35|    Новосибирск|
|   Маша| 32|Санкт-Петербург|
|  Артем| 23|         Москва|
|Виталий| 35|      Махачкала|
|   Маша| 18|         Казань|
|   Олег| 28|         Самара|
|   Вася| 25|         Казань|
|   Петя| 35|    Новосибирск|
|  Елена| 40|         Москва|
+-------+---+---------------+



In [13]:
# Посчитаем средний возраст по городам
agg_df = df_data.groupBy("city").agg(F.avg("age").alias("age_avg"))

agg_df.show()

+---------------+-------+
|           city|age_avg|
+---------------+-------+
|         Москва|   26.5|
|         Казань|   21.5|
|    Новосибирск|   35.0|
|Санкт-Петербург|   32.0|
|      Махачкала|   35.0|
+---------------+-------+



In [14]:
pivot_by_age_city = df_data.groupBy("city").pivot("age").agg(F.count("name"))

print("Результат pivot():")
pivot_by_age_city.show()

print("Результат pivot() с заполнением NULL значений нулями:")
pivot_by_age_city_filled = pivot_by_age_city.na.fill(0)
pivot_by_age_city_filled.show()

Результат pivot():
+---------------+----+----+----+----+----+----+
|           city|  18|  23|  25|  30|  32|  35|
+---------------+----+----+----+----+----+----+
|         Москва|NULL|   1|NULL|   1|NULL|NULL|
|         Казань|   1|NULL|   1|NULL|NULL|NULL|
|      Махачкала|NULL|NULL|NULL|NULL|NULL|   1|
|Санкт-Петербург|NULL|NULL|NULL|NULL|   1|NULL|
|    Новосибирск|NULL|NULL|NULL|NULL|NULL|   1|
+---------------+----+----+----+----+----+----+

Результат pivot() с заполнением NULL значений нулями:
+---------------+---+---+---+---+---+---+
|           city| 18| 23| 25| 30| 32| 35|
+---------------+---+---+---+---+---+---+
|         Москва|  0|  1|  0|  1|  0|  0|
|         Казань|  1|  0|  1|  0|  0|  0|
|      Махачкала|  0|  0|  0|  0|  0|  1|
|Санкт-Петербург|  0|  0|  0|  0|  1|  0|
|    Новосибирск|  0|  0|  0|  0|  0|  1|
+---------------+---+---+---+---+---+---+



In [15]:
drop_df = df_data.drop("age")
drop_df.show()

+-------+---------------+
|   name|           city|
+-------+---------------+
|  Алиса|         Москва|
|   Вася|         Казань|
|   Петя|    Новосибирск|
|   Маша|Санкт-Петербург|
|  Артем|         Москва|
|Виталий|      Махачкала|
|   Маша|         Казань|
+-------+---------------+



In [16]:
total_rows = df_data.count()
print(f"Общее количество строк: {total_rows}")
cnt_masha = df_data.filter(df_data["name"] == "Маша").count()
print(f"Количество с именем Маша: {cnt_masha}")

Общее количество строк: 7
Количество с именем Маша: 2


In [17]:
all_data_list = df_data.collect()
print(all_data_list)


[Row(name='Алиса', age=30, city='Москва'), Row(name='Вася', age=25, city='Казань'), Row(name='Петя', age=35, city='Новосибирск'), Row(name='Маша', age=32, city='Санкт-Петербург'), Row(name='Артем', age=23, city='Москва'), Row(name='Виталий', age=35, city='Махачкала'), Row(name='Маша', age=18, city='Казань')]


In [18]:
pandas_df = df_data.toPandas()
print(pandas_df)


      name  age             city
0    Алиса   30           Москва
1     Вася   25           Казань
2     Петя   35      Новосибирск
3     Маша   32  Санкт-Петербург
4    Артем   23           Москва
5  Виталий   35        Махачкала
6     Маша   18           Казань


In [19]:
print(df_data.take(2))

[Row(name='Алиса', age=30, city='Москва'), Row(name='Вася', age=25, city='Казань')]


In [20]:
# Указываем путь для сохранения
output_path = "4/example_data_csv" 

# .mode("overwrite") позволяет перезаписать данные, если директория уже существует.
# header=True добавляет заголовок в файл.
df_data.write.mode("overwrite").csv(output_path, header=True)